In [12]:
from sqlalchemy import create_engine, Column, Integer, Float, String, Boolean, ForeignKey, func
from sqlalchemy.orm import sessionmaker, relationship, declarative_base

# Define the base class
Base = declarative_base()

In [13]:
# Define ORM classes
class Vintage(Base):
    __tablename__ = 'vintages'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    wine_id = Column(Integer, ForeignKey('wines.id'))
    ratings_average = Column(Float)
    ratings_count = Column(Integer)
    year = Column(Integer)
    price_euros = Column(Float)
    price_discounted_from = Column(Float)
    price_discount_percentage = Column(Float)
    bottle_volume_ml = Column(Integer)

class Wine(Base):
    __tablename__ = 'wines'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    is_natural = Column(Boolean)
    region_id = Column(Integer, ForeignKey('regions.id'))
    winery_id = Column(Integer, ForeignKey('wineries.id'))
    ratings_average = Column(Float)
    ratings_count = Column(Integer)
    url = Column(String)
    acidity = Column(Float)
    fizziness = Column(Float)
    intensity = Column(Float)
    sweetness = Column(Float)
    tannin = Column(Float)
    user_structure_count = Column(Integer)

class Country(Base):
    __tablename__ = 'countries'
    code = Column(String, primary_key=True)
    name = Column(String)
    regions_count = Column(Integer)
    users_count = Column(Integer)
    wines_count = Column(Integer)
    wineries_count = Column(Integer)
    # Relationships
    regions = relationship("Region", backref="country")
    top_lists = relationship("TopList", backref="country")
    most_used_grapes = relationship("MostUsedGrapes", backref="country")

class Region(Base):
    __tablename__ = 'regions'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    country_code = Column(String, ForeignKey('countries.code'))
    # Relationship
    wines = relationship("Wine", backref="region")

class Winery(Base):
    __tablename__ = 'wineries'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    # Relationship
    wines = relationship("Wine", backref="winery")

class Grape(Base):
    __tablename__ = 'grapes'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    wines_count = Column(Integer)
    # Relationship
    most_used_grapes = relationship("MostUsedGrapes", backref="grape")

class TopList(Base):
    __tablename__ = 'toplists'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    country_code = Column(String, ForeignKey('countries.code'))

class VintageTopListRanking(Base):
    __tablename__ = 'vintage_toplists_rankings'
    id = Column(Integer, primary_key=True)
    top_list_id = Column(Integer, ForeignKey('toplists.id'))
    vintage_id = Column(Integer, ForeignKey('vintages.id'))
    rank = Column(Integer)
    previous_rank = Column(Integer)

class MostUsedGrapes(Base):
    __tablename__ = 'most_used_grapes_per_country'
    id = Column(Integer, primary_key=True)
    country_code = Column(String, ForeignKey('countries.code'))
    grape_id = Column(Integer, ForeignKey('grapes.id'))

class Keyword(Base):
    __tablename__ = 'keywords'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    # Relationship
    keywords_wine = relationship("KeywordsWine", backref="keyword")

class KeywordsWine(Base):
    __tablename__ = 'keywords_wine'
    id = Column(Integer, primary_key=True)
    keyword_type = Column(String)
    count = Column(Integer)
    keyword_id = Column(Integer, ForeignKey('keywords.id'))
    wine_id = Column(Integer, ForeignKey('wines.id'))
    group_name = Column(String)

In [14]:
# Create engine and bind session
engine = create_engine('sqlite:///vivino.db')


# Now, reflect the tables to make sure the definitions are correct and all relationships are established.
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

In [15]:
# highlight 10 wines based on ratings and number of ratings (likely popular and well-regarded by customers)
top_wines = (session.query(Wine)
             .filter(Wine.ratings_count > 50)  # Only consider wines with more than 50 ratings
             .order_by(Wine.ratings_average.desc())  # Order by average rating
             .limit(10)  # Limit to top 10
             .all())

for wine in top_wines:
    print(f"Wine ID: {wine.id}, Name: {wine.name}, Average Rating: {wine.ratings_average}, Ratings Count: {wine.ratings_count}")

Wine ID: 1187886, Name: Amarone della Valpolicella Classico Riserva, Average Rating: 4.8, Ratings Count: 587
Wine ID: 1611255, Name: Cabernet Sauvignon, Average Rating: 4.8, Ratings Count: 2941
Wine ID: 5806244, Name: Mágico, Average Rating: 4.8, Ratings Count: 146
Wine ID: 6534388, Name: Fratini Bolgheri Superiore, Average Rating: 4.8, Ratings Count: 153
Wine ID: 7266631, Name: Cristal Rosé Vinothèque, Average Rating: 4.8, Ratings Count: 88
Wine ID: 8023, Name: IX Estate Red, Average Rating: 4.7, Ratings Count: 2810
Wine ID: 66294, Name: Special Selection Cabernet Sauvignon, Average Rating: 4.7, Ratings Count: 41236
Wine ID: 77136, Name: Unico Reserva Especial Edición, Average Rating: 4.7, Ratings Count: 13025
Wine ID: 77137, Name: Unico, Average Rating: 4.7, Ratings Count: 45140
Wine ID: 77178, Name: Eszencia, Average Rating: 4.7, Ratings Count: 488


In [16]:
# Assuming uniqueness can be determined by a wine being the only one from a specific grape or region
unique_wines = (session.query(Wine.name, Region.name)
                .join(Region, Wine.region_id == Region.id)  # Join with Region table
                .group_by(Wine.region_id)
                .having(func.count(Wine.id) == 1)
                .limit(10)
                .all())

for wine_name, region_name in unique_wines:
    print(f"Wine Name: {wine_name}, Region: {region_name}")

Wine Name: Evenstad Reserve Pinot Noir, Region: Willamette Valley
Wine Name: Belles Soeurs Cuvée Pinot Noir, Region: Yamhill County
Wine Name: Rapture Cabernet Sauvignon, Region: Lodi
Wine Name: Howell Mountain Cabernet Sauvignon, Region: Howell Mountain
Wine Name: One Point Five Cabernet Sauvignon, Region: Stags Leap District
Wine Name: Blueline Estate Cabernet Sauvignon, Region: Calistoga
Wine Name: Hyde Vineyard Chardonnay, Region: Los Carneros
Wine Name: Bentrock Vineyard Chardonnay, Region: Sta. Rita Hills
Wine Name: Alexander Valley Cabernet Sauvignon, Region: Alexander Valley
Wine Name: Cabernet Sauvignon, Region: Columbia Valley
